<a href="https://colab.research.google.com/github/SarahGoddaer/Machine_Learning_course_UGent_D012554_kaggle/blob/master/9%3A%20Voting_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn import metrics

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Here, I want to know if combining the different results of the different models I used can make an improvement towards predicting test set labels.

#Voting classifier

In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")
testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")

features = trainset.copy()
features.pop('label')
feature_names = list(features.columns)

test_features = testset.copy()
test_features.pop('index')
test_feature_names = list(test_features.columns)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

# Training classifiers
clf1 = make_pipeline(StandardScaler(), SVC(C=10,gamma=1, probability=True))
clf2 = RandomForestClassifier(criterion='entropy', n_estimators=250, 
                              max_depth=30, random_state=0)
clf3 = XGBClassifier(n_estimators= 900, gamma=0, max_depth=7, 
                     min_child_weight=0, learning_rate=0.1, 
                     subsample=0.85, colsample_bytree=0.9)
clf4 = KNeighborsClassifier(n_neighbors=4, weights='distance')
clf5 = make_pipeline(StandardScaler(),LogisticRegression(C=100))

eclf = VotingClassifier(estimators=[('svc', clf1), ('rf', clf2), 
                                    ('xgb', clf3), ('kkn', clf4), 
                                    ('reg', clf5)], voting='soft')


In [0]:
for clf, name in zip([clf1, clf2, clf3, clf4, clf5, eclf], 
                      ['SVC', 'Random Forest', 'XGBoost', 'KNeighbors', 
                       'Logistic regression', 'Ensemble']):
  scores = cross_val_score(clf, features, trainset.label, 
                           scoring='roc_auc', cv=10)
  print("AUC score: %0.5f (+/- %0.5f) [%s]" % (scores.mean(), scores.std(), name))

AUC score: 0.96253 (+/- 0.01182) [SVC]
AUC score: 0.93205 (+/- 0.01971) [Random Forest]
AUC score: 0.94602 (+/- 0.02002) [XGBoost]
AUC score: 0.95938 (+/- 0.01119) [KNeighbors]
AUC score: 0.65963 (+/- 0.03643) [Logistic regression]
AUC score: 0.96706 (+/- 0.01412) [Ensemble]


Ok, this seems to work! Now I will see if this approach maybe better works without the logistic regression model...

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

# Training classifiers
clf1 = make_pipeline(StandardScaler(), SVC(C=10,gamma=1, probability=True))
clf2 = RandomForestClassifier(criterion='entropy', n_estimators=250, 
                              max_depth=30, random_state=0)
clf3 = XGBClassifier(n_estimators= 900, gamma=0, max_depth=7, 
                     min_child_weight=0, learning_rate=0.1, 
                     subsample=0.85, colsample_bytree=0.9)
clf4 = KNeighborsClassifier(n_neighbors=4, weights='distance')

eclf = VotingClassifier(estimators=[('svc', clf1), ('rf', clf2), 
                                    ('xgb', clf3), ('kkn', clf4)],
                        voting='soft')

In [0]:
for clf, name in zip([clf1, clf2, clf3, clf4, eclf], 
                      ['SVC', 'Random Forest', 'XGBoost', 'KNeighbors', 
                       'Ensemble']):
  scores = cross_val_score(clf, features, trainset.label, 
                           scoring='roc_auc', cv=10)
  print("AUC score: %0.5f (+/- %0.5f) [%s]" % (scores.mean(), scores.std(), name))

AUC score: 0.96253 (+/- 0.01182) [SVC]
AUC score: 0.93205 (+/- 0.01971) [Random Forest]
AUC score: 0.94602 (+/- 0.02002) [XGBoost]
AUC score: 0.95938 (+/- 0.01119) [KNeighbors]
AUC score: 0.96951 (+/- 0.01216) [Ensemble]


So it seems to be that this also gives a better outcome on the trainset if you compare the AUC score. This is almost as good as what I achieved with stacking.

In [0]:
score = cross_val_score(eclf, features, trainset.label, scoring = 'roc_auc', cv=10).mean()
print('AUC score: ' + str(score))

AUC score: 0.9694883699379112


I put some weights in it, just to try and see what influence this has on the AUC score.

In [0]:
votingClass = VotingClassifier(estimators=[('svc', clf1), ('rf', clf2), 
                                    ('xgb', clf3), ('kkn', clf4)],
                        voting='soft', weights=[2,1, 1, 2])

for clf, name in zip([clf1, clf2, clf3, clf4, votingClass], 
                      ['SVC', 'Random Forest', 'XGBoost', 'KNeighbors', 
                       'Ensemble']):
  scores = cross_val_score(clf, features, trainset.label, 
                           scoring='roc_auc', cv=10)
  print("AUC score: %0.5f (+/- %0.5f) [%s]" % (scores.mean(), scores.std(), name))

AUC score: 0.96253 (+/- 0.01182) [SVC]
AUC score: 0.93205 (+/- 0.01971) [Random Forest]
AUC score: 0.94602 (+/- 0.02002) [XGBoost]
AUC score: 0.95938 (+/- 0.01119) [KNeighbors]
AUC score: 0.97152 (+/- 0.01101) [Ensemble]


Well I think this was luck with choosing my weights... But this score is the same as with the stacking technique, apart from a tiny difference.

In [0]:
from sklearn.model_selection import GridSearchCV

vc = VotingClassifier(estimators=[('svc', clf1), ('rf', clf2), 
                                    ('xgb', clf3), ('kkn', clf4)],
                        voting='soft')

params = {'weights':[[2,1,1,2],[3,1,2,3],[3,1,2,2]]}
grid_Search = GridSearchCV(param_grid = params, estimator=vc, scoring='roc_auc', cv=10)
grid_Search.fit(features, trainset.label)
print(grid_Search.best_params_)
print(grid_Search.best_score_)

{'weights': [2, 1, 1, 2]}
0.9715493629713814


In [0]:
from sklearn.model_selection import GridSearchCV

vc = VotingClassifier(estimators=[('svc', clf1), ('rf', clf2), 
                                    ('xgb', clf3), ('kkn', clf4)],
                        voting='soft')

params = {'weights':[[2,1,1,2],[3,3,1,3],[2,2,1,2]]}
grid_Search = GridSearchCV(param_grid = params, estimator=vc, scoring='roc_auc', cv=10)
grid_Search.fit(features, trainset.label)
print(grid_Search.best_params_)
print(grid_Search.best_score_)

{'weights': [2, 1, 1, 2]}
0.9715089976282636


In [0]:
from sklearn.model_selection import GridSearchCV

vc = VotingClassifier(estimators=[('svc', clf1), ('rf', clf2), 
                                    ('xgb', clf3), ('kkn', clf4)],
                        voting='soft')

params = {'weights':[[2,1,1,2],[3,1,1,3],[2,1,2,3], [3,1,3,3]]}
grid_Search = GridSearchCV(param_grid = params, estimator=vc, scoring='roc_auc', cv=10)
grid_Search.fit(features, trainset.label)
print(grid_Search.best_params_)
print(grid_Search.best_score_)

{'weights': [3, 1, 1, 3]}
0.9718729435701912


This is the best score I have 'till now! Maybe I could tune those weights a bit more, we will see..

In [0]:
from sklearn.model_selection import GridSearchCV

vc = VotingClassifier(estimators=[('svc', clf1), ('rf', clf2), 
                                    ('xgb', clf3), ('kkn', clf4)],
                        voting='soft')

params = {'weights':[[3,1,1,3],[4,1,1,4],[2,1,2,3], [3,1,3,3]]}
grid_Search = GridSearchCV(param_grid = params, estimator=vc, scoring='roc_auc', cv=10)
grid_Search.fit(features, trainset.label)
print(grid_Search.best_params_)
print(grid_Search.best_score_)

{'weights': [3, 1, 1, 3]}
0.9719032272518513


In [0]:
params = {'weights':[[3,1,1,3],[3,1,1,2],[2,1,1,3], [3,1,1,4],[4,1,1,3]]}
grid_Search = GridSearchCV(param_grid = params, estimator=vc, scoring='roc_auc', cv=10)
grid_Search.fit(features, trainset.label)
print(grid_Search.best_params_)
print(grid_Search.best_score_)

{'weights': [4, 1, 1, 3]}
0.9720850454336694


Well I think I have done enough for today and I'm pretty satisfied about the results that I obtained toda. Everything that is left is just make predictions with this model with the weights: [4,1,1,3].

In [0]:
model = grid_Search.best_estimator_
score = cross_val_score(model, features, trainset.label,scoring='roc_auc', cv= 10).mean()
print('AUC score for trainset: '+ str(score))

AUC score for trainset: 0.9719941363427603


In [0]:
model.fit(features,trainset.label)
predictions = model.predict_proba(test_features)[:,1]

sample_submission = pd.DataFrame({'index': testset['index'], 'label': predictions})
sample_submission.head()

,index,label
0,0,0.011382
1,1,0.453319
2,2,0.014171
3,3,0.992859
4,4,0.959147


In [0]:
filename = "my_prediction_results6_real.csv"
sample_submission.to_csv(filename,index=False)

This was until Sunday my best performing model.. Monday, new week, new chances, so I take this day as a last opportunity to make an even more better performing model. I've updated the other models to a slightly higher level, by choosing other hyper parameters. Let's see if the weighted average of these better models also can make a difference in performance... Hopefully a better performing model will result from this.

In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression

# Training classifiers
clf1 = make_pipeline(StandardScaler(), SVC(C=8, gamma=1, probability=True))
clf2 = RandomForestClassifier(criterion='entropy', n_estimators=250, 
                              max_depth=30, random_state=0, bootstrap=False)
clf3 = XGBClassifier(n_estimators=900, gamma=0, max_depth=8, min_child_weight=0, 
                      learning_rate=0.1, colsample_bytree=0.9, subsample=0.6,
                      reg_alpha=1e-5)
clf4 = KNeighborsClassifier(algorithm='auto', leaf_size=1, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2.5,
                     weights='distance')

ensemble = VotingClassifier(estimators=[('svc', clf1), ('rf', clf2), 
                                    ('xgb', clf3), ('kkn', clf4)]
                          , voting='soft')

In [7]:
for clf, name in zip([clf1, clf2, clf3, clf4, ensemble], 
                      ['SVC', 'Random Forest', 'XGBoost', 'KNeighbors', 
                       'Ensemble']):
  scores = cross_val_score(clf, features, trainset.label, 
                           scoring='roc_auc', cv=20)
  print("AUC score: %0.5f (+/- %0.5f) [%s]" % (scores.mean(), scores.std(), name))

AUC score: 0.96336 (+/- 0.01755) [SVC]
AUC score: 0.93676 (+/- 0.02884) [Random Forest]
AUC score: 0.94625 (+/- 0.02750) [XGBoost]
AUC score: 0.96126 (+/- 0.01561) [KNeighbors]
AUC score: 0.97035 (+/- 0.01747) [Ensemble]


In [8]:
from sklearn.model_selection import GridSearchCV

params = {'weights':[[2,1,1,2],[3,1,1,3],[4,1,1,3],[4,1,1,4]]}
grid_Search = GridSearchCV(param_grid = params, estimator=ensemble, 
                           scoring='roc_auc', cv=20)
grid_Search.fit(features, trainset.label)
print(grid_Search.best_params_)
print(grid_Search.best_score_)

{'weights': [4, 1, 1, 3]}
0.9727945395988874
